# 0. Libraries

In [9]:
# Pandas tools
import pandas as pd
import pandas as pd
import pandas.io.sql as sqlio
from pandas import DataFrame

# Database-related tools
import duckdb
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine, text, inspect
from sqlalchemy.orm import sessionmaker

# Other utilities
import os

# 1. Connection to the databases (including the data warehouse) and the .csv

Connection or extraction to the provided data sources. 

In [6]:
# Connection to the AIMS and AMOS databases
if 'aims' in globals():
  aims.close()
aims = psycopg2.connect(database='aims', user='bse_airlines', host='dtim.essi.upc.edu', password='BSE2024!', options='-c search_path=bda-aims')
if 'amos' in globals():
  amos.close()
amos = psycopg2.connect(database='amos', user='bse_airlines', host='dtim.essi.upc.edu', password='BSE2024!', options='-c search_path=bda-amos')

# Reading the .csv (must ensure that it is in the same working directory as this notebook)
df_manufacturer_lookup = pd.read_csv('aircraft-manufaturerinfo-lookup.csv')

Creation of the data warehouse (ROLAP) using the SQL scripts in the `tables_dw` folder. Note that the code below creates a new database for the specified user.

In [12]:
# Database connection parameters
DB_NAME = "lab3_dw_delgado_fernandez"
DB_USER = "postgres"    # Adjust as necessary
DB_PASSWORD = "datamg"    # Adjust as necessary
DB_HOST = "localhost"    # Adjust as necessary
DB_PORT = "5432"    # Adjust as necessary

# PostgreSQL default connection (to 'postgres' database)
DEFAULT_DB_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/postgres"

# Function to drop and recreate the database
def recreate_database():
    try:
        # Connect to the default 'postgres' database
        connection = psycopg2.connect(
            dbname="postgres", 
            user=DB_USER, 
            password=DB_PASSWORD, 
            host=DB_HOST, 
            port=DB_PORT
        )
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = connection.cursor()

        # Drop the database if it exists
        cursor.execute(f"SELECT pg_terminate_backend(pg_stat_activity.pid) "
                       f"FROM pg_stat_activity WHERE pg_stat_activity.datname = '{DB_NAME}';")
        cursor.execute(f"DROP DATABASE IF EXISTS {DB_NAME};")
        print(f"Database '{DB_NAME}' dropped successfully (if it existed).")

        # Create the new database
        cursor.execute(f"CREATE DATABASE {DB_NAME};")
        print(f"Database '{DB_NAME}' created successfully.")

        # Close the connection
        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Error recreating database: {e}")

# Function to execute an SQL file
def execute_sql_file(file_path, engine):
    with open(file_path, 'r') as file:
        sql_code = file.read()
        with engine.connect() as connection:
            try:
                connection.execute(text(sql_code))
                connection.commit()  # Commit the transaction
                print(f"Executed SQL file: {file_path}")
            except Exception as e:
                print(f"Error executing {file_path}: {e}")

# Function to drop all tables in the database
def drop_all_tables(engine):
    inspector = inspect(engine)
    with engine.connect() as connection:
        transaction = connection.begin()
        try:
            for table_name in inspector.get_table_names():
                connection.execute(text(f"DROP TABLE IF EXISTS {table_name} CASCADE"))
            transaction.commit()
            print("Dropped all existing tables.")
        except Exception as e:
            transaction.rollback()
            print(f"Error dropping tables: {e}")

# Function to create tables
def create_tables(engine):
    directory = 'tables_dw'  # Directory containing SQL files
    files_path = sorted([os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.sql')])

    for file_path in files_path:
        execute_sql_file(file_path, engine)

if __name__ == "__main__":
    # Step 1: Recreate the database (i.e., drop preexisting DB with the same name 
    # and create a new one)
    recreate_database()

    # Step 2: Connect to the newly created database
    DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(DATABASE_URL)

    # Step 3: Drop all tables and create new ones
    drop_all_tables(engine)
    create_tables(engine)

Database 'lab3_dw_delgado_fernandez' dropped successfully (if it existed).
Database 'lab3_dw_delgado_fernandez' created successfully.
Dropped all existing tables.
Executed SQL file: tables_dw/01_Route.sql
Executed SQL file: tables_dw/02_Aircraft.sql
Executed SQL file: tables_dw/03_Flights.sql
Executed SQL file: tables_dw/04_Scheduled_routes.sql
Executed SQL file: tables_dw/05_Flight_Issues.sql
Executed SQL file: tables_dw/06_Maintenance_time.sql
Executed SQL file: tables_dw/07_ADOS.sql


It is possible to connect to this database through DBeaver after it was been created by creating a new PostgreSQL connection and specifying the adequate fields *name*, *user*, *password*, *host* and *port*.

In [ ]:
# Connection to the data warehouse
outputDB = duckdb.connect(database=':memory:')

## Explore the AIMS data source (cursor)

Here you can run differnet queries in order to explore the database tables, their columns and the values stored inside.
In this case, we load the result set of our SQL query into a cursor object over which we can iterate in order to obtain its values.

In [ ]:
#Write any query you want to explore the database!
# limit X is used to reduce the output to only top X rows. Useful in the case of large DB tables.

Q1 = "SELECT * FROM Flights limit 10;"

# Creatingo a cursor and executing a query (DO NOT MODIFY!)
cursor = aims.cursor()
cursor.execute(Q1)
result = cursor.fetchall()
# printing out the results
for row in result:
    print(row)

cursor.close()

(1, 'XY-RJL', datetime.datetime(2023, 8, 3, 11, 3, 3, 875940), datetime.datetime(2023, 8, 3, 14, 3, 3, 875940), 'Flight         ', '230803-NRN-JMK-9129-XY-RJL', 'NRN', 'JMK', None, None, True, None, 98, 3, 2)
(2, 'XY-OZE', datetime.datetime(2023, 7, 26, 14, 50, 15, 569812), datetime.datetime(2023, 7, 26, 15, 50, 15, 569812), 'Flight         ', '230726-HAU-SAW-9867-XY-OZE', 'HAU', 'SAW', None, None, True, None, 107, 4, 3)
(3, 'XY-SJZ', datetime.datetime(2023, 11, 20, 9, 1, 4, 308766), datetime.datetime(2023, 11, 20, 13, 1, 4, 308766), 'Flight         ', '231120-HER-VAA-6975-XY-SJZ', 'HER', 'VAA', None, None, True, None, 137, 3, 2)
(4, 'XY-OXK', datetime.datetime(2023, 6, 13, 9, 55, 2, 480695), datetime.datetime(2023, 6, 13, 11, 55, 2, 480695), 'Flight         ', '230613-VNO-EGC-9468-XY-OXK', 'VNO', 'EGC', None, None, True, None, 102, 4, 2)
(5, 'XY-DGU', datetime.datetime(2023, 3, 12, 0, 41, 45, 198279), datetime.datetime(2023, 3, 12, 3, 41, 45, 198279), 'Flight         ', '230312-BLL-BV

## Explore the AMOS data source (data frame)

Here you can run differnet queries in order to explore the database tables, their columns and the values stored inside.

In this case, we load the result of the query into a data frame structure, over which we can then apply any vector/data frame operation.

In [ ]:
#Write any query you want to explore the database!
# limit X is used to reduce the output to only top X rows. Useful in the case of large DB tables.

Q2 = "SELECT * FROM WorkOrders LIMIT 10;"

dfWO = sqlio.read_sql_query(Q2, amos)
dfWO.head(10)


<ipython-input-5-b8fca4c891ac>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfWO = sqlio.read_sql_query(Q2, amos)


,workorderid,aircraftregistration,executiondate,executionplace,workpackage,kind
0,2,XY-AAB,2023-01-01,FAO,1.0,Forecast
1,3,XY-AAB,2023-01-01,FAO,1.0,Forecast
2,10,XY-AAB,2023-01-04,MUC,4.0,Forecast
3,11,XY-AAB,2023-01-04,MUC,4.0,Forecast
4,14,XY-AAB,2023-01-05,BJV,NaN,Forecast
5,16,XY-AAB,2023-01-07,AJA,5.0,Forecast
6,18,XY-AAB,2023-01-09,BRE,NaN,Forecast
7,22,XY-AAB,2023-01-14,bhD,7.0,Forecast
8,23,XY-AAB,2023-01-14,bhD,7.0,Forecast
9,32,XY-AAB,2023-01-17,ZTH,11.0,Forecast


## Read the CSV lookup file

Here, we read the content of the CSV file and load it into a data frame.  


Select CSV file from the disk.
**Important:** Execute this only once to upload the CSV file into your Google Collab storage and then use it as many times as you want to load the data into a Data Frame.

In [ ]:
uploaded = files.upload()

Saving aircraft-manufaturerinfo-lookup.csv to aircraft-manufaturerinfo-lookup.csv


Load the CSV file into a data frame and print out its content.

In [ ]:
dfAircrafts = pd.read_csv(io.BytesIO(uploaded['aircraft-manufaturerinfo-lookup.csv']))
dfAircrafts.head(100)

,aircraft_reg_code,manufacturer_serial_number,aircraft_model,aircraft_manufacturer
0,XY-ZCK,MSN 8143,777,Airbus
1,XY-UWV,MSN 3747,777,Airbus
2,XY-HWJ,MSN 9579,747,Airbus
3,XY-NIP,MSN 4982,747,Boeing
4,XY-CMS,MSN 2662,A319,Airbus
...,...,...,...,...
95,XY-FOI,MSN 8688,A350 XWB,Airbus
96,XY-NIX,MSN 1785,A319,Airbus
97,XY-ULG,MSN 8396,737,Boeing
98,XY-DOZ,MSN 7544,A319,Airbus


##Join data frames

Here we can join WorkOrders data frame with the Aircraft lookup dataframe and add new columns like aircraft model and manufacturer to the WorkOrders data frame.

For this purpose we can use either **join** or **merge** operation in Pandas.  

Join operation requires setting up the indexes of the data frames using the keys on which we want to do join.
More info on join operation in Pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html

Merge operation can perform joins using any specified column of the data frame. *Here we use a merge operation* and join the two data frames of the **aircraftregistration** and **aircraft_reg_code**
More info on merge operation in Pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html


In [ ]:
dfJoin = dfWO.merge(dfAircrafts, left_on='aircraftregistration', right_on='aircraft_reg_code')


dfJoin.head(1000)

,workorderid,aircraftregistration,executiondate,executionplace,workpackage,kind,aircraft_reg_code,manufacturer_serial_number,aircraft_model,aircraft_manufacturer
0,2,XY-AAB,2023-01-01,FAO,1.0,Forecast,XY-AAB,MSN 9818,767,Boeing
1,3,XY-AAB,2023-01-01,FAO,1.0,Forecast,XY-AAB,MSN 9818,767,Boeing
2,10,XY-AAB,2023-01-04,MUC,4.0,Forecast,XY-AAB,MSN 9818,767,Boeing
3,11,XY-AAB,2023-01-04,MUC,4.0,Forecast,XY-AAB,MSN 9818,767,Boeing
4,14,XY-AAB,2023-01-05,BJV,NaN,Forecast,XY-AAB,MSN 9818,767,Boeing
5,16,XY-AAB,2023-01-07,AJA,5.0,Forecast,XY-AAB,MSN 9818,767,Boeing
6,18,XY-AAB,2023-01-09,BRE,NaN,Forecast,XY-AAB,MSN 9818,767,Boeing
7,22,XY-AAB,2023-01-14,bhD,7.0,Forecast,XY-AAB,MSN 9818,767,Boeing
8,23,XY-AAB,2023-01-14,bhD,7.0,Forecast,XY-AAB,MSN 9818,767,Boeing
9,32,XY-AAB,2023-01-17,ZTH,11.0,Forecast,XY-AAB,MSN 9818,767,Boeing


##Filtering data

Here we can filter the data of our data frame by selecting some of the values of the columns. For instance, we can choose only the workorders that had execution dates between 2023-10-19 and 2023-10-23.

More info on loc operation in Pandas:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html

In [ ]:
dfFiltered = dfJoin.loc[(dfJoin['executiondate'] >= pd.to_datetime('2023-10-19').date()) & (dfJoin['executiondate'] <= pd.to_datetime('2023-10-23').date())]

dfFiltered.head(1000)

,workorderid,aircraftregistration,executiondate,executionplace,workpackage,kind,aircraft_reg_code,manufacturer_serial_number,aircraft_model,aircraft_manufacturer
159,388,XY-AAB,2023-10-19,SPC,107.0,Forecast,XY-AAB,MSN 9818,767,Boeing
160,393,XY-AAB,2023-10-22,TSE,109.0,Forecast,XY-AAB,MSN 9818,767,Boeing
161,394,XY-AAB,2023-10-22,TSE,109.0,Forecast,XY-AAB,MSN 9818,767,Boeing
162,395,XY-AAB,2023-10-23,SXF,110.0,Forecast,XY-AAB,MSN 9818,767,Boeing
163,396,XY-AAB,2023-10-23,IEv,111.0,Forecast,XY-AAB,MSN 9818,767,Boeing
426,956,XY-ACY,2023-10-20,ALA,NaN,Forecast,XY-ACY,MSN 4159,737,Boeing
427,957,XY-ACY,2023-10-20,ALA,NaN,Forecast,XY-ACY,MSN 4159,737,Boeing
428,959,XY-ACY,2023-10-22,KIV,251.0,Forecast,XY-ACY,MSN 4159,737,Boeing
429,960,XY-ACY,2023-10-22,KIV,251.0,Forecast,XY-ACY,MSN 4159,737,Boeing
430,963,XY-ACY,2023-10-23,MUC,253.0,Forecast,XY-ACY,MSN 4159,737,Boeing


##Loading data into a Database table

Lastly, we can load the resulting data frame into a database table.

More info on inporting data to  DuckDB from Pandas:
https://duckdb.org/docs/guides/python/import_pandas.html

In [ ]:
# We first store the schema and content of our dataframe into a temp DB table.
outputDB.execute("CREATE TABLE IF NOT EXISTS temp AS SELECT * FROM dfFiltered")


# We can then read the stored DB table into a new data frame and print its content
outputDB.execute("select * from temp").df()

###Finalize work

Once you finish working with the notebook, please execute the code below to close the connection with the database.

In [ ]:
aims.close()
amos.close()

NameError: name 'aims' is not defined